# 1. Train test split

In [1]:
import sys
sys.path.append('../../')
import pandas as pd
from SynTemp.SynUtils.utils import load_database, save_database, train_val_test_split_df
data = pd.DataFrame(load_database('../../Data/DPO/USPTO_50K/USPTO_50K_aam_reactions.json.gz'))
original_data = pd.read_csv('../../Data/USPTO_50K/USPTO_50K_original.csv')

data = pd.concat([data, original_data['class']], axis=1)

In [ ]:
train_df, val_df, test_df = train_val_test_split_df(data, target='class', random_state=42)

save_database(train_df.to_dict('records'), '../../Data/DPO/USPTO_50K/train.json.gz')
save_database(val_df.to_dict('records'), '../../Data/DPO/USPTO_50K/val.json.gz')
save_database(test_df.to_dict('records'), '../../Data/DPO/USPTO_50K/test.json.gz')

# 2. Rules Writing

In [4]:
#folder_name = 'Non_hydrogen'
#folder_name = 'Good_hydrogen'
folder_name = 'Hydrogen'

In [6]:
from SynTemp.SynUtils.utils import load_from_pickle, save_to_pickle
from SynTemp.SynUtils.graph_utils import add_child_ids 
temp = load_from_pickle(f'../../Data/DPO/USPTO_50K/{folder_name}/templates.pkl.gz')
processed_data = add_child_ids(temp)
save_to_pickle(processed_data, f'../../Data/DPO/USPTO_50K/{folder_name}/hier_rules.pkl.gz')

In [9]:
from SynTemp.SynUtils.utils import stratified_random_sample
from SynTemp.SynRule.rule_writing import RuleWriting
data = load_from_pickle(f'../../Data/DPO/USPTO_50K/{folder_name}/cluster.pkl.gz')
for i in range(0,4):
    samples = stratified_random_sample(data, f'Cluster_R{i}', 1, 42)
    if folder_name == 'Good_hydrogen':
        samples = [value for value in samples if value['Reaction Type'] in ['Single Cyclic', 'Complex Cyclic']]
    rules = RuleWriting.auto_extraction(data_dicts=samples, id_column=f'Cluster_R{i}',
                                        save_path=f'../../Data/DPO/USPTO_50K/{folder_name}/R{i}/')